In [21]:
import requests
import pandas as pd

from datetime import datetime

from openai import OpenAI
from bs4 import BeautifulSoup

client = OpenAI(
    api_key= "" # PUT YOUR API KEY here
)

In [22]:
# # Luis Suarez
player_name = 'Luis Suarez'
url = 'https://fbref.com/en/players/a6154613/Luis-Suarez'
attrs = 'scout_summary_AM'

# Lamine Yamal
#player_name = 'Lamine Yamal'
#url = 'https://fbref.com/en/players/82ec26c1/Lamine-Yamal'
#attrs = 'scout_summary_AM'

# # Nathan Broadhead
#player_name = 'Nathan Broadhead'
#url = 'https://fbref.com/en/players/43309491/Nathan-Broadhead'
#attrs = 'scout_summary_AM'

# Attempt to fetch the tables from the URL
try:
    # Get all tables from the webpage
    tables = pd.read_html(url)
    
    # Print all tables to help identify the correct one
    for i, table in enumerate(tables):
        print(f"Table {i}:")
        print(table.head(), "\n")
    
    # Manually inspect and select the correct table by index
    df = tables[0]  # Adjust this index based on the printed output
    
    print(df)
except ValueError as e:
    print(f"Error reading the HTML: {e}")
except Exception as e:
    print(f"An error occurred: {e}")
    

    # Attempt to fetch the specific table with the given attributes
try:
    df = pd.read_html(url, attrs={'id': attrs})[0]
    print(df)
except ValueError as e:
    print(f"Error reading the HTML with specified attributes: {e}")
except Exception as e:
    print(f"An error occurred: {e}")


#df = pd.read_html(
#    url, 
#    attrs={'id': attrs} # Change the id & class to correspond to the table
#)[0]


Table 0:
                  Statistic Per 90  Percentile
0         Non-Penalty Goals   0.68        94.0
1      npxG: Non-Penalty xG   0.48        86.0
2               Shots Total   3.74        92.0
3                   Assists   0.41        99.0
4  xAG: Exp. Assisted Goals   0.23        96.0 

Table 1:
   Rk              Player   Nation        Squad  Compare
0   1    Tarik Tissoudali   ma MAR         Gent  Compare
1   2     Viktor Gyökeres   se SWE  Sporting CP  Compare
2   3  Andreas Hountondji   bj BEN     Rodez AF  Compare
3   4      Adam Armstrong  eng ENG  Southampton  Compare
4   5           Che Adams  sct SCO  Southampton  Compare 

Table 2:
  Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0 Unnamed: 3_level_0  \
                Date                Day              Round              Venue   
0         2024-05-18                Sat     Regular Season               Home   
1         2024-05-15                Wed     Regular Season               Away   
2         2024-05-11 

In [23]:
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [24]:
# Get the player's age, team, league, and position
position = soup.select_one('p:-soup-contains("Position")').text.split(':')[-2].split(' ')[0].strip()
birthday = soup.select_one('span[id="necro-birth"]').text.strip()

# get the player's age
age = (datetime.now() - datetime.strptime(birthday, '%B %d, %Y')).days // 365
team = soup.select_one('p:-soup-contains("Club")').text.split(':')[-1].strip()

In [25]:
# df.columns = df.columns.droplevel(0)

df = df.dropna(subset='Statistic')

In [26]:
prompt = f"""

I need you to create a scouting report on {player_name}. Can you provide me with a summary of their strengths and weaknesses?

Here is the data I have on him:

Player: {player_name}
Position: {position}
Age: {age}
Team: {team}

{df.to_markdown()}


Return the scouting report in the following markdown format:

# Scouting Report for {player_name}

## Strengths
< a list of 1 to 3 strengths >

## Weaknesses
< a list of 1 to 3 weaknesses >

## Summary
< a brief summary of the player's overall performance and if he would be beneficial to the team >
"""

In [27]:
print(prompt)



I need you to create a scouting report on Luis Suarez. Can you provide me with a summary of their strengths and weaknesses?

Here is the data I have on him:

Player: Luis Suarez
Position: FW
Age: 37
Team: Inter Miami

|    | Statistic                | Per 90   |   Percentile |
|---:|:-------------------------|:---------|-------------:|
|  0 | Non-Penalty Goals        | 0.68     |           94 |
|  1 | npxG: Non-Penalty xG     | 0.48     |           86 |
|  2 | Shots Total              | 3.74     |           92 |
|  3 | Assists                  | 0.41     |           99 |
|  4 | xAG: Exp. Assisted Goals | 0.23     |           96 |
|  5 | npxG + xAG               | 0.71     |           92 |
|  6 | Shot-Creating Actions    | 4.15     |           97 |
|  8 | Passes Attempted         | 33.17    |           97 |
|  9 | Pass Completion %        | 70.6%    |           63 |
| 10 | Progressive Passes       | 3.44     |           95 |
| 11 | Progressive Carries      | 1.00     |           41 |


In [28]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a professional football (soccer) scout."},
        {"role": "user", "content": prompt},
    ],
)

In [29]:
print(response.choices[0].message.content)

# Scouting Report for Luis Suarez

## Strengths
1. **Goal Scoring Ability**: With a Non-Penalty Goals rate of 0.68 per 90 minutes (94th percentile), Suarez remains an elite goal scorer, consistently finding the back of the net.
2. **Creativity and Playmaking**: Suarez’s Assists (0.41 per 90, 99th percentile) and xAG (0.23 per 90, 96th percentile) demonstrate his ability to create goal-scoring opportunities for his teammates, augmenting his role as a forward.
3. **Shot-Creating Actions**: Averaging 4.15 shot-creating actions per 90 minutes (97th percentile), Suarez excels in generating offensive plays and opportunities, making him a dynamic threat in the attack.

## Weaknesses
1. **Defensive Contributions**: Suarez's Tackles (0.22 per 90, 4th percentile), Interceptions (0.14 per 90, 35th percentile), and Blocks (0.11 per 90, 1st percentile) stats reveal his minimal impact on the defensive end.
2. **Aerial Duels**: With only 0.62 Aerials Won per 90 minutes (6th percentile), Suarez is not

In [30]:
# save the scouting report to a new markdown file
with open(f"/Users/rcamacho/Documents/{player_name}_scouting_report.md", "w") as f:
    f.write(response.choices[0].message.content)